<a href="https://colab.research.google.com/github/MatthewRomanishin/courses/blob/main/matmech/mathmech_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [20]:
import numpy as np
import pandas as pd
import dateutil

#**Подфрейм Вандермонда**

Для вектора $x=(x_0, \ldots, x_n)$ матрица Вандермона выглядит следующим образом:
$$A = \begin{pmatrix}
1 & x_0 & \cdots & x_0^n \\
1 & x_1 & \cdots & x_1^n \\
\vdots & \ddots & \ddots & \vdots \\
1 & x_n & \cdots &  x_n^n
\end{pmatrix}$$ 

А датафрейм Вандермонда выглядит аналогично:

|       | 0 | 1  |  |n |
|-------|---|----|--|----|
|0      | 1 | $x_0$ | $\ldots$ | $x_0^n$ |
|1      | 1 | $x_1$ | $\ldots$ | $x_1^n$ |
|$\vdots$ | $\vdots$ | $\ddots$ | $\ddots$ | $\vdots$ |
|$n$  | 1 |$x_n$|  | $x_n^n$ |


На вход подается вектор вещественных чисел длины $n+1 > 1$ и две пары индексов. Вам нужно вернуть Фрейм Вандермонда и **под**Фрейм Вандермонда, где первая пара индексов - задает срез по строкам, а вторая пара индексов - срез по колонкам. 

Подсказка: не нужно создавать самим матрицу Вандермонда, она давно реализована за вас в numpy: [np.vander](https://docs.scipy.org/doc/numpy/reference/generated/numpy.vander.html)

#### Input:
```python
x = np.array([3,1,5,4,2])
indices = (0,3)
columns = (2,4)
```
#### **под**Фрейм Вандермонда:

|  | 2 | 3  
|--|---|----
|0 | 9 | 27  
|1 | 1 | 1   
|2 | 25 |125

In [3]:
x = np.array([3,1,5,4,2])
indexes = (0,3)
columns = (2,4)

In [4]:
def subVander(x: np.ndarray, indices: tuple, columns: tuple) -> (pd.DataFrame, pd.DataFrame):
    vander = pd.DataFrame(np.vander(x, increasing=True))    
    subvander = vander.iloc[indices[0]:indices[1],columns[0]:columns[1]]
    return subvander

In [5]:
subVander(x, indexes, columns)

,2,3
0,9,27
1,1,1
2,25,125


#**Бухгалтерия зоопарка**

Вам на вход подается словарь, где ключ - это тип животного, а значение - словарь с признаками этого животного, где ключ - тип признака, а значение - значение признака (Типичный json проще говоря). Наименования признаков животного - всегда строки. Значения признаков - любой из типов pandas.

Вам следует создать табличку, где по строчкам будут идти животные, а по колонкам - их признаки, которая удовлетворяет следующим условиям:

Тип животного нужно выделить в отдельную колонку Type
Строки отсортированы по типу животного в алфавитном порядке
Колонки отсортированы в алфавитном порядке, кроме колонки Type - она первая
Индексы строк - ряд натуральных чисел начиная с 0 без пропусков
Имейте в виду, что признаки у двух животных могут не совпадать, значит незаполненные данные нужно заполнить Nan значением.

Верните на выходе табличку(DataFrame), в которой отсутствуют Nan значения. При этом могут отсутствовать некоторые признаки, но животные должны присутствовать все. Изначальные типы значений из словаря: int64, float64, bool и.т.д. должны сохраниться и в конечной табличке, а не превратиться в object-ы. (От удаляемых признаков этого, очевидно, не требуется).

#### Input:
```python
ZOO = {
        'cat':{'color':'black', 'tail_len': 50, 'injured': False}, 
        'dog':{'age': 6, 'tail_len': 30.5, 'injured': True}
      }
```
#### Output:
||Type|injured|tail_len|
|-|-|-|-|


In [6]:
ZOO = {
        'cat':{'color':'black', 'tail_len': 50, 'injured': False}, 
        'dog':{'age': 6, 'tail_len': 30.5, 'injured': True}
      }


In [7]:
def ZOOtable(zoo: dict) -> pd.DataFrame:
    df = pd.DataFrame.from_dict(zoo, orient='index')
    df['Type']=zoo.keys()
    df = df.sort_index(axis = 0)
    df = df.sort_index(axis = 1)
    df = df.reset_index(drop=True)
    df = df.dropna(axis=1)
    return df

In [8]:
ZOOtable(ZOO)

,Type,injured,tail_len
0,cat,False,50.0
1,dog,True,30.5


#**Простые преобразования**

На вход подается DataFrame из 3-х колонок дата рождения и смерти человека на русском языке в формате представленом ниже:

| |      Имя        | Дата рождения  |   Дата смерти   |
|-|-----------------|----------------|-----------------|
|0|Никола Тесла     |10 июля 1856 г. |7 января 1943 г. |
|1|Альберт Эйнштейн |14 марта 1879 г.|18 апреля 1955 г.|

Необходимо вернуть исходную таблицу с добавленным в конце столбцом полных лет жизни.

| |       Имя        | Дата рождения  |   Дата смерти   |Полных лет|
|-|:-----------------|:---------------|:----------------|:---------|
|0|Никола Тесла      |10 июля 1856 г. |7 января 1943 г. |86        |
|1|Альберт Эйнштейн  |14 марта 1879 г.|18 апреля 1955 г.|76        |

In [9]:
df = pd.DataFrame([["Никола Тесла", "10 июля 1856 г.", "7 января 1943 г."],
              ["Альберт Эйнштейн", "14 марта 1879 г.", "18 апреля 1955 г."]],
             columns=["Имя", "Дата рождения", "Дата смерти"])
df

,Имя,Дата рождения,Дата смерти
0,Никола Тесла,10 июля 1856 г.,7 января 1943 г.
1,Альберт Эйнштейн,14 марта 1879 г.,18 апреля 1955 г.


In [10]:
def rus_feature(df: pd.DataFrame) -> pd.DataFrame:
    date_of_birth = df['Дата рождения'].apply(lambda x: x.split()[:3])
    date_of_dead = df['Дата смерти'].apply(lambda x: x.split()[:3])

    x = {'января':'01',
     'февраля':'02',
     'марта':'03',
     'апреля':'04',
     'мая':'05',
     'июня':'06',
     'июля':'07',
     'августа':'08',
     'сентября':'09',
     'октября':'10',
     'ноября':'11',
     'декабря':'12'}

    month1 = date_of_birth.apply(lambda y: x.get(y[1]))
    month2 = date_of_dead.apply(lambda y: x.get(y[1]))

    day1 = date_of_birth.apply(lambda x: x[0])
    day2 = date_of_dead.apply(lambda x: x[0])

    year1 = date_of_birth.apply(lambda x: x[2])
    year2 = date_of_dead.apply(lambda x: x[2])

    date1 = day1+month1+year1
    date2 = day2+month2+year2

    date1 = pd.to_datetime(date1, format='%d%m%Y')
    date2 = pd.to_datetime(date2, format='%d%m%Y')

    df['date1'] = date1
    df['date2'] = date2

    df['Полных лет'] = df.apply(lambda x: dateutil.relativedelta.relativedelta(x['date2'], x['date1']).years, axis=1)
    df.drop(['date1', 'date2'], axis='columns', inplace=True)

    return df

In [11]:
rus_feature(df)

,Имя,Дата рождения,Дата смерти,Полных лет
0,Никола Тесла,10 июля 1856 г.,7 января 1943 г.,86
1,Альберт Эйнштейн,14 марта 1879 г.,18 апреля 1955 г.,76


In [12]:
df = pd.read_csv('/content/gdrive/My Drive/Introduction to DS/data/titanic_train.csv')

#**Характеристики**
В этой задаче на вход подаются всем известные данные (titanic_train.csv).

Верните среднее, медиану, максимальное и минимальное значение возраста погибших мужчин. Именно в данном порядке.

In [13]:
def men_stat(df: pd.DataFrame) -> (float, float, float, float):
    death_men = df[(df['Sex'] == 'male') & (df['Survived'] == 0)]['Age']
    stat = death_men.agg(['mean','median','max','min']).values
    return list(stat)

In [14]:
men_stat(df)

[31.618055555555557, 29.0, 74.0, 1.0]

#**Свобдная таблица**
В этой задаче на вход подаются всем известные данные(titanic_train.csv).

Сделать сводную таблицу по медианному возрасту для пола и класса. Для примера посмотрите сводную таблицу по сумме выживших, для пола и класса.

In [16]:
def age_stat(df: pd.DataFrame) -> pd.DataFrame:
    stat = df.groupby(['Sex','Pclass'])[['Age']].median()
    return stat

In [17]:
age_stat(df)

Age
Sex    Pclass      
female 1       35.0
       2       28.0
       3       21.5
male   1       40.0
       2       30.0
       3       25.0

#**Популярные девушки**
В этой задаче на вход подаются всем известные данные(titanic_train.csv).

Выведите список имен незамужних женщин(Miss) отсортированный по популярности.

В полном имени девушек имя - это первое слово без скобок после Miss.
Остальные строки не рассматриваем.
Девушки с одинаковой популярностью сортируются по имени в алфавитном порядке.
Слово/имя - подстрока без пробелов. Популярность - количество таких имен в таблице.

In [21]:
def fename_stat(df: pd.DataFrame) -> pd.DataFrame:
    df = df[df['Name']!='Meanwell, Miss. (Marion Ogden)']
    name = df.loc[df['Sex']=='female','Name']

    miss = name.str.extract(r"Miss\.\s+(\w+)", expand=False)
    miss.dropna(inplace=True)
    val_idx = miss.value_counts()

    df = pd.DataFrame({'Name':val_idx.index, 'Popularity':val_idx.values})
    df.sort_values(['Popularity', 'Name'], ascending=[False, True], inplace=True) 
    df.reset_index(inplace=True)

    return df.iloc[:,1:]

In [22]:
fename_stat(df)

,Name,Popularity
0,Anna,9
1,Mary,9
2,Margaret,6
3,Elizabeth,5
4,Alice,4
...,...,...
115,Thamine,1
116,Torborg,1
117,Velin,1
118,Virginia,1
